# Switch Eager / Graph Execution

* [tf.executing_eagerly](https://www.tensorflow.org/api_docs/python/tf/executing_eagerly)

> Checks whether the current thread has eager execution enabled.

* [tf.config.run_functions_eagerly](https://www.tensorflow.org/api_docs/python/tf/config/run_functions_eagerly)

> Enables / disables eager execution of tf.functions.

## Graph Mode

Cannot use:

* for/while -> Use ```tf.while_loop``` or ```tf.py_function```.
* if -> Use ```tf.cond``` or ```tf.py_function```.
* print -> Use ```tf.print```
* assert
* tf.Tensor.shape attribute -> Use ```tf.shape()```
* tf.Tensor.ndim attribute

In [1]:
import tensorflow as tf

In [2]:
@tf.function
def my_func(a):
    print("Python side effect")
    return a + a

a_fn = tf.function(my_func)

# Confirm Eager Model

In [3]:
tf.executing_eagerly()

True

# Eager Execution

In [11]:
tf.config.run_functions_eagerly(True)

In [12]:
tf.executing_eagerly()

True

In [13]:
for i in range(3):
    a_fn(tf.constant(1))

Python side effect
Python side effect
Python side effect


# Graph Execution

In [14]:
tf.config.run_functions_eagerly(False)

In [7]:
tf.executing_eagerly()

True

In [8]:
for i in range(3):
    a_fn(tf.constant(1))

Python side effect


2023-03-05 10:52:28.541052: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
